In [1]:
from bs4 import BeautifulSoup
import csv
import selenium

In [16]:
def extract_links(html_content):
    # Parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(html_content, 'lxml')  # You can also use 'html.parser' if you don't want to use lxml
    
    # Find all the <a> tags in the HTML content
    links = soup.find_all('a')

    # Extract the href attribute (the URL) from each <a> tag
    urls = [link.get('href') for link in links if link.get('href') is not None]

    return urls

def get_html(url):
    response = requests.get(url)
    return response.text

def read_html_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    return html_content

def fetch_element_text_with_selenium(url):
    # Set up Chrome options
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Runs Chrome in headless mode.

    # Set path to chromedriver as per your configuration
    service = Service(ChromeDriverManager().install())

    # Initialize the driver
    driver = webdriver.Chrome(service=service, options=chrome_options)

    # Open the page
    driver.get(url)

    nameXpath = '/html/body/main/div/article/section/div/div/div/div/table/tbody/tr[1]/td[2]'
    nameElement = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, nameXpath))
    )
    name = nameElement.text
    
    adressXpath = '/html/body/main/div/article/section/div/div/div/div/table/tbody/tr[4]/td[2]'
    adressElement = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, adressXpath))
    )
    adress = adressElement.text
    
    ownerXpath = '/html/body/main/div/article/section/div/div/div/div/table/tbody/tr[5]/td[2]'
    ownerElement = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, ownerXpath))
    )
    owner = ownerElement.text
    
    iframe = driver.find_element(By.ID, 'gmap-preview')
    driver.switch_to.frame(iframe)
    
    coordinatesXpath = '//*[@id="mapDiv"]/div/div/div[4]/div/div/div/div/div[1]/div[1]'
    coordinatesElement = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, coordinatesXpath))
    )
    coordinates = coordinatesElement.text

    # Clean up: close the browser
    driver.quit()

    return [name, adress, owner, coordinates]

In [6]:
filepath = "source.html"
html = read_html_file(filepath)
urls = extract_links(html)

In [12]:
place_urls = []
for url in urls:
    if "place" in url:
        place_urls.append(url)

In [82]:
place_data = []
i = 0
for url in place_urls:
    listOfValues = fetch_element_text_with_selenium(url)
    place_data.append(listOfValues)
    i += 1

0 https://www.cervenekontejnery.cz/vyhledavani/?place=44
1 https://www.cervenekontejnery.cz/vyhledavani/?place=52
2 https://www.cervenekontejnery.cz/vyhledavani/?place=3648
3 https://www.cervenekontejnery.cz/vyhledavani/?place=39
4 https://www.cervenekontejnery.cz/vyhledavani/?place=1040808
5 https://www.cervenekontejnery.cz/vyhledavani/?place=1778179
6 https://www.cervenekontejnery.cz/vyhledavani/?place=21725
7 https://www.cervenekontejnery.cz/vyhledavani/?place=582
8 https://www.cervenekontejnery.cz/vyhledavani/?place=576
9 https://www.cervenekontejnery.cz/vyhledavani/?place=50
10 https://www.cervenekontejnery.cz/vyhledavani/?place=3530
11 https://www.cervenekontejnery.cz/vyhledavani/?place=613
12 https://www.cervenekontejnery.cz/vyhledavani/?place=597
13 https://www.cervenekontejnery.cz/vyhledavani/?place=3890
14 https://www.cervenekontejnery.cz/vyhledavani/?place=3687
15 https://www.cervenekontejnery.cz/vyhledavani/?place=3531
16 https://www.cervenekontejnery.cz/vyhledavani/?place=

In [135]:
for i in range(100):
    place_data[i][1] = place_data[i][1].replace('\n',', ')
    place_data[i][3] = place_data[i][3].replace('"', '')
    place_data[i][0] = place_data[i][0].replace(',', ';')
    place_data[i][1] = place_data[i][1].replace(',', ';')
    place_data[i][2] = place_data[i][2].replace(',', ';')

In [137]:
# The path to the output CSV file
file_path = "cervena_data.csv"

# Writing to the CSV file
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Write all the rows at once
    writer.writerows(place_data)